<a href="https://colab.research.google.com/github/anshika0601/ai_voice_assistant/blob/main/ai_voice_assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import os
import tempfile
import whisper
from pydub import AudioSegment
from gtts import gTTS
from dotenv import load_dotenv
from google.colab import userdata
import gradio as gr
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage

# ------------------ Environment ------------------
load_dotenv()
GROQ_API_KEY = userdata.get("GROQ_API_KEY")
if not GROQ_API_KEY:
    raise ValueError("GROQ_API_KEY not found in Colab Secrets. Please add it.")

# ------------------ Whisper ------------------
whisper_model = whisper.load_model("medium")

def convert_to_wav(audio_path: str) -> str:
    wav_path = tempfile.mktemp(suffix=".wav")
    audio = AudioSegment.from_file(audio_path)
    audio = audio.set_frame_rate(16000).set_channels(1)
    audio.export(wav_path, format="wav")
    return wav_path

def transcribe(audio_path: str) -> str:
    audio_path = convert_to_wav(audio_path)
    result = whisper_model.transcribe(audio_path)
    return result["text"]

# ------------------ Groq LLM ------------------
llm = ChatGroq(
    groq_api_key=GROQ_API_KEY,
    model_name="llama-3.1-8b-instant",
    temperature=0.2
)

def generate_reply(user_text: str) -> str:
    resp = llm.invoke([HumanMessage(content=user_text)])
    return resp.content

# ------------------ TTS ------------------
def speak(text: str) -> str:
    out_path = tempfile.mktemp(suffix=".mp3")
    tts = gTTS(text=text, lang="hi")   # Hindi (can change to 'en' for English)
    tts.save(out_path)
    return out_path

# ------------------ Pipeline ------------------
def pipeline(audio_path):
    if not audio_path:
        return "⚠️ No audio captured.", "", None
    user_text = transcribe(audio_path)
    reply_text = generate_reply(user_text)
    reply_audio = speak(reply_text)
    return user_text, reply_text, reply_audio

# ------------------ Gradio UI ------------------
with gr.Blocks(css="""
    .big-title {font-size: 28px; font-weight: bold; text-align: center;}
    .subtitle {font-size: 18px; color: #555; text-align: center; margin-bottom: 20px;}
    .chatbox {background: #f9f9f9; border-radius: 12px; padding: 12px; margin-bottom: 8px;}
""", title="🎙️ AI Voice Assistant") as demo:

    gr.Markdown("<div class='big-title'>🤖 AI Voice Assistant</div>")
    gr.Markdown("<div class='subtitle'>Powered by Whisper + Groq + GTTS</div>")

    with gr.Row():
        with gr.Column(scale=2):
            mic = gr.Audio(
                sources=["microphone"],
                type="filepath",
                label="🎤 Tap to Record",
            )

            with gr.Accordion("📜 Conversation", open=True):
                user_tb = gr.Textbox(label="🧑 You Said", interactive=False)
                reply_tb = gr.Textbox(label="🤖 Assistant Replied", interactive=False)

            speaker = gr.Audio(type="filepath", label="🔊 Assistant Voice Reply")

            mic.change(fn=pipeline, inputs=[mic], outputs=[user_tb, reply_tb, speaker])

        with gr.Column(scale=1):
            gr.Markdown(
                """
                ### 📝 How to Use
                1. **Click the microphone** 🎤 and speak clearly.
                2. The assistant will:
                   - 📝 Transcribe your voice
                   - 💡 Generate a smart reply (Groq LLM)
                   - 🔊 Speak the reply aloud using GTTS
                3. Speak in **English for best accuracy**.
                """
            )

if __name__ == "__main__":
    demo.launch()


100%|█████████████████████████████████████| 1.42G/1.42G [00:40<00:00, 38.1MiB/s]


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9131853eddc5873e23.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [2]:
!pip install openai-whisper gtts pydub gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 8.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 9.5 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=e514faf81d849115dbccbb81ddce902c9b61f487e2b5fabf7c0ef821b0940089
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
Successfully built openai-whisper
  Attempting uninstall: click
    Found existing installation: click 8.2.1
    Uninstalling click-8.2.1:
      Successfully uninstalled click-8.2.1


In [4]:
!pip install langchain langchain-core

In [6]:
!pip install langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 3.2 MB/s eta 0:00:00
